# Public Notebook
https://www.kaggle.com/cdeotte/rapids-svr-boost-17-8

In [ ]:
# based on the post here: https://www.kaggle.com/c/petfinder-pawpularity-score/discussion/275094

import sys
sys.path.append("../input/tez-lib/")
timm_path = "../input/timm-pytorch-image-models/pytorch-image-models-master"
import sys
sys.path.append(timm_path)
import timm

import tez
import albumentations
import pandas as pd
import cv2
import numpy as np
import timm
import random
import torch.nn as nn
from sklearn import metrics
import torch
from tez.callbacks import EarlyStopping
from tqdm import tqdm
import math
import os

class args:
    batch_size = 16
    image_size = 384
    
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [ ]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        
set_seed(42)

In [ ]:
class PawpularDataset:
    def __init__(self, image_paths, dense_features, targets, augmentations):
        self.image_paths = image_paths
        self.dense_features = dense_features
        self.targets = targets
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        image = cv2.imread(self.image_paths[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
            
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        
        features = self.dense_features[item, :]
        targets = self.targets[item]
        
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "features": torch.tensor(features, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.float),
        }
    
class PawpularModel(tez.Model):
    def __init__(self, model_name):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False, in_chans=3)
        self.model.head = nn.Linear(self.model.head.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(140, 64)
        self.dense2 = nn.Linear(64, 1)

    def forward(self, image, features, targets=None):
        x1 = self.model(image)
        x = self.dropout(x1)
        x = torch.cat([x, features], dim=1)
        x = self.dense1(x)
        x = self.dense2(x)
        
        x = torch.cat([x, x1, features], dim=1)
        return x, 0, {}
    
test_aug = albumentations.Compose(
    [
        albumentations.Resize(args.image_size, args.image_size, p=1),
        albumentations.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        albumentations.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        albumentations.RandomBrightnessContrast(p=0.5),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

# Import RAPIDS

In [ ]:
import cuml, pickle
from cuml.svm import SVR
print('RAPIDS version',cuml.__version__,'\n')

LOAD_SVR_FROM_PATH = '../input/svr-models-10-folds/'

df = pd.read_csv('../input/same-old-creating-folds/train_10folds.csv')
print('Train shape:', df.shape )
df.head()

In [ ]:
import cuml, pickle
from cuml.svm import SVR

In [ ]:
super_final_predictions = []
super_final_predictions2 = []
super_final_oof_predictions = []
super_final_oof_predictions2 = []
super_final_oof_true = []

for fold_ in range(10):
    print('#'*25)
    print('### FOLD',fold_+1)
    print('#'*25)
    
    model = PawpularModel(model_name="swin_large_patch4_window12_384")
    model.load(f"../input/paw-models/model_f{fold_}.bin", device="cuda", weights_only=True)

    df_test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
    test_img_paths = [f"../input/petfinder-pawpularity-score/test/{x}.jpg" for x in df_test["Id"].values]
        
    df_valid = df[df.kfold == fold_].reset_index(drop=True)#.iloc[:160]
    valid_img_paths = [f"../input/petfinder-pawpularity-score/train/{x}.jpg" for x in df_valid["Id"].values]

    dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
    ]
    
    name = f"SVR_fold_{fold_}.pkl" 
    if LOAD_SVR_FROM_PATH is None:
        ##################
        # EXTRACT TRAIN EMBEDDINGS
        
        df_train = df[df.kfold != fold_].reset_index(drop=True)
        train_img_paths = [f"../input/petfinder-pawpularity-score/train/{x}.jpg" for x in df_train["Id"].values]
        
        train_dataset = PawpularDataset(
            image_paths=train_img_paths,
            dense_features=df_train[dense_features].values,
            targets=df_train['Pawpularity'].values/100.0,
            augmentations=test_aug,
        )
        print('Extracting train embedding...')
        train_predictions = model.predict(train_dataset, batch_size=2*args.batch_size, n_jobs=-1)
        
    
        embed = np.array([]).reshape((0,128+12))
        for preds in train_predictions:
            embed = np.concatenate([embed,preds[:,1:]],axis=0)
        
        ##################
        # FIT RAPIDS SVR
        print('Fitting SVR...')
        clf = SVR(C=20.0)

        clf.fit(embed.astype('float32'), df_train.Pawpularity.values.astype('int32'))
    
        ##################
        # SAVE RAPIDS SVR 
        pickle.dump(clf, open(name, "wb"))
        
    else:
        ##################
        # LOAD RAPIDS SVR 
        print('Loading SVR...',LOAD_SVR_FROM_PATH+name)
        clf = pickle.load(open(LOAD_SVR_FROM_PATH+name, "rb"))

    ##################
    # TEST PREDICTIONS
    test_dataset = PawpularDataset(
        image_paths=test_img_paths,
        dense_features=df_test[dense_features].values,
        targets=np.ones(len(test_img_paths)),
        augmentations=test_aug,
    )
    print('Predicting test...')
    test_predictions = model.predict(test_dataset, batch_size=2*args.batch_size, n_jobs=-1)
    #print(test_predictions.shape)

    final_test_predictions = []
    embed = np.array([]).reshape((0,128+12))
    for preds in test_predictions: #tqdm
        final_test_predictions.extend(preds[:,:1].ravel().tolist())
        embed = np.concatenate([embed,preds[:,1:]],axis=0)

    

    final_test_predictions = [sigmoid(x) * 100 for x in final_test_predictions]
    final_test_predictions2 = clf.predict(embed)
    super_final_predictions.append(final_test_predictions)
    super_final_predictions2.append(final_test_predictions2)
    

In [ ]:
# FORCE SVR WEIGHT TO LOWER VALUE TO HELP PUBLIC LB
best_w = 0.5

In [ ]:
super_final_predictions = np.mean(np.column_stack(super_final_predictions), axis=1)
super_final_predictions2 = np.mean(np.column_stack(super_final_predictions2), axis=1)
df_test["Pawpularity"] = (1-best_w)*super_final_predictions + best_w*super_final_predictions2
df_test = df_test[["Id", "Pawpularity"]]


In [ ]:
df_test.head()

# Our models

In [ ]:
timm_path = "../input/timm-pytorch-image-models/pytorch-image-models-master"
import sys
sys.path.append(timm_path)
import timm
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2

import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset,DataLoader
from torch import optim
import shutil
from shutil import copyfile
import random
import PIL
from PIL import Image
import warnings
warnings.filterwarnings('ignore')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
test_dir = "../input/petfinder-pawpularity-score/test"
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

# YOLO detection

In [ ]:
if not os.path.exists('/kaggle/working/yolov5s-master'):
    shutil.copytree('/kaggle/input/yolov5s-master', '/kaggle/working/yolov5x6')

# Can't figure out how to disable the font download in torchhub, so I manually copy it over to satisfy the check in plot.py
os.mkdir('/root/.config/Ultralytics/')
copyfile('/kaggle/input/yolov5-model/Arial.ttf', '/root/.config/Ultralytics/Arial.ttf')

In [ ]:
# Use torch hub to load our model locally
model = torch.hub.load('/kaggle/working/yolov5x6', 'custom', path='/kaggle/input/yolov5-model/yolov5x6.pt', source='local')
model.max_det = 1
model.classes = [15,16]
model.multi_label = False

In [ ]:
for index, row in test.iterrows():
    img = Image.open(test_dir + "/" + row['Id'] + ".jpg")
    imgs = [img]
    results = model(imgs, size=1280)
    results.crop(save=True)
    results.save()
    df = results.pandas().xyxy[0]
    
    pet_class = "unknown"
    if len(df) > 0:
        pet_class = df['name'].values[0]
          
    test.loc[test['Id'] == row['Id'], 'pet_class'] = pet_class
    
test.head(10)

In [ ]:
test['path'] = [test_dir +'/' + x + '.jpg' for x in test["Id"].values]

In [ ]:
shutil.rmtree('./runs')
shutil.rmtree('./yolov5x6')

In [ ]:
test.head()

# Dividing data into cats/dogs

In [ ]:
cats = test[test.pet_class == 'cat'].reset_index(drop=True)
dogs_unk = test[test.pet_class != 'cat'].reset_index(drop=True)

# Models


In [ ]:
class Model1(nn.Module):
    def __init__(self,pretrained):
        super().__init__()
        self.backbone = timm.create_model("tf_efficientnet_b4_ns", pretrained=pretrained, num_classes=0, drop_rate=0., drop_path_rate=0.,global_pool='')
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc3_A = nn.Linear(1792,12)
        self.fc3_B = nn.Linear(1792,1)
    
    def forward(self,image):
        image = self.backbone(image)
        
        if(len(image.shape) == 4):#for efficientnet models
            image = self.pool(image)
            image = image.view(image.shape[0], -1)

        dec2 = self.fc3_B(image)
        dec1 = self.fc3_A(image)
        return image , dec2

In [ ]:
class Model2(nn.Module):
    def __init__(self,pretrained):
        super().__init__()
        self.backbone = timm.create_model('swin_base_patch4_window12_384_in22k', pretrained=pretrained, num_classes=0, drop_rate=0., drop_path_rate=0.,global_pool='')
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc3_A = nn.Linear(1024,12)
        self.fc3_B = nn.Linear(1024,1)
    
    def forward(self,x):
        x = self.backbone(x)
        dec2 = self.fc3_B(x)
        dec1 = self.fc3_A(x)
        return x,dec2

In [ ]:
class Model3(nn.Module):
    def __init__(self,pretrained):
        super().__init__()
        self.backbone = timm.create_model("swin_large_patch4_window7_224", pretrained=False, num_classes=0, drop_rate=0.0, drop_path_rate=0.0,global_pool='')
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc3_A = nn.Linear(1536,12)
        self.fc3_B = nn.Linear(1536,1)
        self.do = nn.Dropout(p=0.3)
    
    def forward(self,image):
        image = self.backbone(image)
        
        if(len(image.shape) == 4):#for efficientnet models
            image = self.pool(image)
            image = image.view(image.shape[0], -1)

        dec2 = self.fc3_B(image)
      
        return image , dec2

In [ ]:
class Model4(nn.Module):
    def __init__(self,pretrained):
        super().__init__()
        self.backbone = timm.create_model("swin_large_patch4_window12_384", pretrained=False, num_classes=0, drop_rate=0.0, drop_path_rate=0.0,global_pool='')
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc3_A = nn.Linear(1536,12)
        self.fc3_B = nn.Linear(1536,1)
        self.do = nn.Dropout(p=0.3)
    
    def forward(self,image):
        image = self.backbone(image)
        
        if(len(image.shape) == 4):#for efficientnet models
            image = self.pool(image)
            image = image.view(image.shape[0], -1)

        dec2 = self.fc3_B(image)
      
        return image , dec2

# Augs

In [ ]:
val_aug2 = A.Compose(
    [ 
        A.Resize(320,320,p=1.0),
        A.CenterCrop(224,224,p=1.0),
        A.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
        ToTensorV2()
    ]
)

In [ ]:
image_size = 384
val_aug1 = A.Compose(
    [ 
        A.Resize(480,480,p=1.0),
        A.CenterCrop(image_size,image_size,p=1.0),

        A.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
        ToTensorV2()
    ]
)

# Dataset class

In [ ]:
class Pets(Dataset):
    def __init__(self , df,augs = None):
        self.df = df
        self.augs = augs
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        img_src = self.df.loc[idx,'path']
        image = cv2.imread(img_src)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        transformed = self.augs(image=image)
        image = transformed['image']
       
        return image

# Inference functions

In [ ]:
def inference_func1(test_loader , path):
    m = Model1(False)
    m = m.to(device)
    m.load_state_dict(torch.load(path))
    m.eval()
    bar = tqdm(test_loader)

    PREDS = []
    
    with torch.no_grad():
        for batch_idx, images in enumerate(bar):
            x = images.to(device)
            _,output = m(x)
            output = output.sigmoid()
            output = output*100
            PREDS += [output.detach().cpu()]
        PREDS = torch.cat(PREDS).cpu().numpy()  
    return PREDS

In [ ]:
def inference_func2(test_loader , path):
    m = Model2(False)
    m = m.to(device)
    m.load_state_dict(torch.load(path))
    m.eval()
    bar = tqdm(test_loader)

    PREDS = []
    
    with torch.no_grad():
        for batch_idx, images in enumerate(bar):
            x = images.to(device)
            _,output = m(x)
            output = output.sigmoid()
            output = output*100
            PREDS += [output.detach().cpu()]
        PREDS = torch.cat(PREDS).cpu().numpy()  
    return PREDS

In [ ]:
def inference_func3(test_loader , path):
    m = Model3(False)
    m = m.to(device)
    m.load_state_dict(torch.load(path))
    m.eval()
    bar = tqdm(test_loader)

    PREDS = []
    
    with torch.no_grad():
        for batch_idx, images in enumerate(bar):
            x = images.to(device)
            _,output = m(x)
            output = output.sigmoid()
            output = output*100
            PREDS += [output.detach().cpu()]
        PREDS = torch.cat(PREDS).cpu().numpy()  
    return PREDS

In [ ]:
dogs_unk_dataset1 = Pets(dogs_unk.reset_index(drop=True),augs = val_aug1)
dogs_unk_loader1 = DataLoader(dogs_unk_dataset1, batch_size=16, shuffle=False,  num_workers=4)

In [ ]:
dogs_unk_dataset2 = Pets(dogs_unk.reset_index(drop=True),augs = val_aug2)
dogs_unk_loader2 = DataLoader(dogs_unk_dataset2, batch_size=16, shuffle=False,  num_workers=4)

# Predicting for !Cats 

In [ ]:
if(len(dogs_unk) > 0):
    yp1  = inference_func1(dogs_unk_loader1,'../input/pawpularity-models-dog-v2/tf_efficientnet_b4_ns - dog - Fold 0 - val rmse 18.9697.pth')
    yp2  = inference_func1(dogs_unk_loader1,'../input/pawpularity-models-dog-v2/tf_efficientnet_b4_ns - dog - Fold 1 - val rmse 19.0007.pth')
    yp3  = inference_func1(dogs_unk_loader1,'../input/pawpularity-models-dog-v2/tf_efficientnet_b4_ns - dog - Fold 2 - val rmse 19.5034.pth')
    yp4  = inference_func1(dogs_unk_loader1,'../input/pawpularity-models-dog-v2/tf_efficientnet_b4_ns - dog - Fold 3 - val rmse 20.1689.pth')
    yp5  = inference_func1(dogs_unk_loader1,'../input/pawpularity-models-dog-v2/tf_efficientnet_b4_ns - dog - Fold 4 - val rmse 19.3915.pth')
    yp6  = inference_func1(dogs_unk_loader1,'../input/pawpularity-models-dog-v2/tf_efficientnet_b4_ns - dog - Fold 5 - val rmse 19.9424.pth')
    yp7  = inference_func1(dogs_unk_loader1,'../input/pawpularity-models-dog-v2/tf_efficientnet_b4_ns - dog - Fold 6 - val rmse 19.6432.pth')
    yp8  = inference_func1(dogs_unk_loader1,'../input/pawpularity-models-dog-v2/tf_efficientnet_b4_ns - dog - Fold 7 - val rmse 20.5341.pth')
    yp9  = inference_func1(dogs_unk_loader1,'../input/pawpularity-models-dog-v2/tf_efficientnet_b4_ns - dog - Fold 8 - val rmse 19.4683.pth')
    yp10 = inference_func1(dogs_unk_loader1,'../input/pawpularity-models-dog-v2/tf_efficientnet_b4_ns - dog - Fold 9 - val rmse 19.4951.pth')
    yp11 = inference_func3(dogs_unk_loader2,'../input/exp1-224-dogs/Fold 0 with val_rmse 18.57408182689726.pth')
    yp12 = inference_func3(dogs_unk_loader2,'../input/exp1-224-dogs/Fold 1 with val_rmse 20.93912210908539.pth')
    yp13 = inference_func3(dogs_unk_loader2,'../input/exp1-224-dogs/Fold 2 with val_rmse 19.520618494757.pth')
    yp14 = inference_func3(dogs_unk_loader2,'../input/exp1-224-dogs/Fold 3 with val_rmse 19.780201070324544.pth')
    yp15 = inference_func3(dogs_unk_loader2,'../input/exp1-224-dogs/Fold 4 with val_rmse 19.5155116100269.pth')
    yp16 = inference_func3(dogs_unk_loader2,'../input/exp1-224-dogs/Fold 5 with val_rmse 20.58098075077962.pth')
    yp17 = inference_func3(dogs_unk_loader2,'../input/exp1-224-dogs/Fold 6 with val_rmse 19.79990146746921.pth')
    yp18 = inference_func3(dogs_unk_loader2,'../input/exp1-224-dogs/Fold 7 with val_rmse 19.779097576099065.pth')
    yp19 = inference_func3(dogs_unk_loader2,'../input/exp1-224-dogs/Fold 8 with val_rmse 19.374419857287354.pth')
    yp20 = inference_func3(dogs_unk_loader2,'../input/exp1-224-dogs/Fold 9 with val_rmse 19.94593150668674.pth')

In [ ]:
if(len(dogs_unk) > 0):
    dogs_unk['Pawpularity'] = (yp1+ yp2+ yp3+yp4+yp5+yp6+yp7+yp8+yp9+yp10+yp11+yp12+yp13+yp14+yp15+yp16+yp17+yp18+yp19+yp20 )/20.

In [ ]:
dogs_unk

# Predicting for images cats

In [ ]:
cats_dataset = Pets(cats.reset_index(drop=True),augs = val_aug1)
cats_loader = DataLoader(cats_dataset, batch_size=16, shuffle=False,  num_workers=4)

In [ ]:
if(len(cats) > 0):
    y_p1 = inference_func1(cats_loader,'../input/paw-models-cat-v2/tf_efficientnet_b4_ns - cat - Fold 0 - val rmse 15.4214.pth')
    y_p2 = inference_func1(cats_loader,'../input/paw-models-cat-v2/tf_efficientnet_b4_ns - cat - Fold 1 - val rmse 14.8935.pth')
    y_p3 = inference_func1(cats_loader,'../input/paw-models-cat-v2/tf_efficientnet_b4_ns - cat - Fold 2 - val rmse 15.8637.pth')
    y_p4 = inference_func1(cats_loader,'../input/paw-models-cat-v2/tf_efficientnet_b4_ns - cat - Fold 3 - val rmse 15.2904.pth')
    y_p5 = inference_func1(cats_loader,'../input/paw-models-cat-v2/tf_efficientnet_b4_ns - cat - Fold 4 - val rmse 14.5136.pth')
    y_p6 = inference_func1(cats_loader,'../input/paw-models-cat-v2/tf_efficientnet_b4_ns - cat - Fold 5 - val rmse 15.7827.pth')
    y_p7 = inference_func1(cats_loader,'../input/paw-models-cat-v2/tf_efficientnet_b4_ns - cat - Fold 6 - val rmse 16.0911.pth')
    y_p8 = inference_func1(cats_loader,'../input/paw-models-cat-v2/tf_efficientnet_b4_ns - cat - Fold 7 - val rmse 15.7253.pth')
    y_p9 = inference_func1(cats_loader,'../input/paw-models-cat-v2/tf_efficientnet_b4_ns - cat - Fold 8 - val rmse 15.3140.pth')
    y_p10 = inference_func1(cats_loader,'../input/paw-models-cat-v2/tf_efficientnet_b4_ns - cat - Fold 9 - val rmse 15.4286.pth')
    y_p11 = inference_func2(cats_loader,'../input/paw-models-swin-cat/swin_base_patch4_window12_384_in22k - cat - Fold 0 - val rmse 14.8155.pth')
    y_p12 = inference_func2(cats_loader,'../input/paw-models-swin-cat/swin_base_patch4_window12_384_in22k - cat - Fold 1 - val rmse 14.4614.pth')
    y_p13 = inference_func2(cats_loader,'../input/paw-models-swin-cat/swin_base_patch4_window12_384_in22k - cat - Fold 2 - val rmse 15.0405.pth')
    y_p14 = inference_func2(cats_loader,'../input/paw-models-swin-cat/swin_base_patch4_window12_384_in22k - cat - Fold 3 - val rmse 14.8688.pth')
    y_p15 = inference_func2(cats_loader,'../input/paw-models-swin-cat/swin_base_patch4_window12_384_in22k - cat - Fold 4 - val rmse 15.6885.pth')
    y_p16 = inference_func2(cats_loader,'../input/paw-models-swin-cat/swin_base_patch4_window12_384_in22k - cat - Fold 5 - val rmse 15.0525.pth')
    y_p17 = inference_func2(cats_loader,'../input/paw-models-swin-cat/swin_base_patch4_window12_384_in22k - cat - Fold 6 - val rmse 14.4531.pth')
    y_p18 = inference_func2(cats_loader,'../input/paw-models-swin-cat/swin_base_patch4_window12_384_in22k - cat - Fold 7 - val rmse 14.9049.pth')
    y_p19 = inference_func2(cats_loader,'../input/paw-models-swin-cat/swin_base_patch4_window12_384_in22k - cat - Fold 8 - val rmse 15.0138.pth')
    y_p20 = inference_func2(cats_loader,'../input/paw-models-swin-cat/swin_base_patch4_window12_384_in22k - cat - Fold 9 - val rmse 14.6707.pth')
    

In [ ]:
if(len(cats) > 0):
    cats['Pawpularity'] = (y_p1+ y_p2+ y_p3+y_p4+y_p5+y_p6+y_p7+y_p8+y_p9+y_p10+y_p11+y_p12+y_p13+y_p14+y_p15+y_p16+y_p17+y_p18+y_p19+y_p20 )/20.

In [ ]:
cats

In [ ]:
test['path'] = [test_dir +'/' + x + '.jpg' for x in test["Id"].values]

# Predicing for "All"

# Effnet

In [ ]:
def inference_funcx(oof , path):
    oof_dataset = Pets(oof.reset_index(drop=True),augs = val_aug1)
    oof_loader = DataLoader(oof_dataset, batch_size=16, shuffle=False,  num_workers=4)
    m = Model1(False)
    m = m.to(device)
    m.load_state_dict(torch.load(path))
    m.eval()
    bar = tqdm(oof_loader)

    PREDS1 = []
    PREDS2 = []
    with torch.no_grad():
        for batch_idx, images in enumerate(bar):
            x = images.to(device)
            output1,output2 = m(x)
            output2 = output2.sigmoid()
            output2 = output2*100
            PREDS1 += [output1.detach().cpu()]
            PREDS2 += [output2.detach().cpu()]
        PREDS1 = torch.cat(PREDS1).cpu().numpy()
        PREDS2 = torch.cat(PREDS2).cpu().numpy()  
    return PREDS1,PREDS2

In [ ]:
path = []
path.append("../input/paw-pl-models/tf_efficientnet_b4_ns - all - Fold 0 - val rmse 17.3108.pth")
path.append('../input/paw-pl-models/tf_efficientnet_b4_ns - all - Fold 1 - val rmse 17.4745.pth')
path.append('../input/paw-pl-models/tf_efficientnet_b4_ns - all - Fold 2 - val rmse 16.9196.pth')
path.append('../input/paw-pl-models/tf_efficientnet_b4_ns - all - Fold 3 - val rmse 17.4784.pth')
path.append('../input/paw-pl-models/tf_efficientnet_b4_ns - all - Fold 4 - val rmse 17.0746.pth')
path.append('../input/paw-pl-models/tf_efficientnet_b4_ns - all - Fold 5 - val rmse 16.9158.pth')
path.append('../input/paw-pl-models/tf_efficientnet_b4_ns - all - Fold 6 - val rmse 17.7258.pth')
path.append('../input/paw-pl-models/tf_efficientnet_b4_ns - all - Fold 7 - val rmse 16.9325.pth')
path.append('../input/paw-pl-models/tf_efficientnet_b4_ns - all - Fold 8 - val rmse 16.7591.pth')
path.append('../input/paw-pl-models/tf_efficientnet_b4_ns - all - Fold 9 - val rmse 17.5080.pth')

In [ ]:
LOAD_SVR_FROM_PATH = '../input/effnet-svm-train/'

In [ ]:
super_final_predictions = []
super_final_predictions2 = []
super_final_oof_predictions = []
super_final_oof_predictions2 = []
super_final_oof_true = []

for fold_ in range(10):
    print('#'*25)
    print('### FOLD',fold_+1)
    print('#'*25)
        
    #model.load(f"../input/paw-models/model_f{fold_}.bin", device="cuda", weights_only=True)

    #df_test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
    #df_test['path'] = [f"../input/petfinder-pawpularity-score/test/{x}.jpg" for x in df_test["Id"].values]
        
    #df_valid = df[df.kfold == fold_]
    #valid_img_paths = [f"../input/petfinder-pawpularity-score/train/{x}.jpg" for x in df_valid["Id"].values]

    dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
    ]
    
    name = f"eff_SVR_fold_{fold_}.pkl" 
    if LOAD_SVR_FROM_PATH is None:
        ##################
        # EXTRACT TRAIN EMBEDDINGS
        
        df_train = df[df.kfold != fold_]
        #train_img_paths = [f"../input/petfinder-pawpularity-score/train/{x}.jpg" for x in df_train["Id"].values]
        
        print('Extracting train embedding...')
        embedx,_ = inference_func(df_train , path[fold_])
        
        ##################
        # FIT RAPIDS SVR
        print('Fitting SVR...')
        clf = SVR(C=20.0)
        clf.fit(embedx.astype('float32'), df_train.Pawpularity.values.astype('int32'))
    
        ##################
        # SAVE RAPIDS SVR 
        pickle.dump(clf, open(name, "wb"))
        
    else:
        ##################
        # LOAD RAPIDS SVR 
        print('Loading SVR...',LOAD_SVR_FROM_PATH+name)
        clf = pickle.load(open(LOAD_SVR_FROM_PATH+name, "rb"))

    ##################
    # TEST PREDICTIONS

    print('Predicting test...')
    embedt,final_test_predictions = inference_funcx(test , path[fold_])

    #final_test_predictions = []
   

    #final_test_predictions = [sigmoid(x) * 100 for x in final_test_predictions]
    final_test_predictions2 = clf.predict(embedt)
    super_final_predictions.append(final_test_predictions)
    super_final_predictions2.append(final_test_predictions2)
    ##################
    
    ##################

In [ ]:
best_w = 0.45
super_final_predictions = np.mean(np.column_stack(super_final_predictions), axis=1)
super_final_predictions2 = np.mean(np.column_stack(super_final_predictions2), axis=1)
test['x'] = (1-best_w)*super_final_predictions + best_w*super_final_predictions2
test.head()

# Swin 384

In [ ]:
#image_size = 384
val_augy =A.Compose(
    [ 
        A.Resize(480,480,p=1.0),
        A.CenterCrop(384,384,p=1.0),
        A.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
        ToTensorV2()
    ]
)

In [ ]:
def inference_funcy(oof , path):
    oof_dataset = Pets(oof.reset_index(drop=True),augs = val_augy)
    oof_loader = DataLoader(oof_dataset, batch_size=16, shuffle=False,  num_workers=4)
    m = Model4(False)
    m = m.to(device)
    m.load_state_dict(torch.load(path))
    m.eval()
    bar = tqdm(oof_loader)

    PREDS1 = []
    PREDS2 = []
    with torch.no_grad():
        for batch_idx, images in enumerate(bar):
            x = images.to(device)
            output1,output2 = m(x)
            output2 = output2.sigmoid()
            output2 = output2*100
            PREDS1 += [output1.detach().cpu()]
            PREDS2 += [output2.detach().cpu()]
        PREDS1 = torch.cat(PREDS1).cpu().numpy()
        PREDS2 = torch.cat(PREDS2).cpu().numpy()  
    return PREDS1,PREDS2

In [ ]:
pathy= []
pathy.append("../input/paw-models-384-large-all/swin_large_patch4_window12_384 - all - Fold 0 - val rmse 17.2610.pth")
pathy.append('../input/paw-models-384-large-all/swin_large_patch4_window12_384 - all - Fold 1 - val rmse 17.2024.pth')
pathy.append('../input/paw-models-384-large-all/swin_large_patch4_window12_384 - all - Fold 2 - val rmse 16.8636.pth')
pathy.append('../input/paw-models-384-large-all/swin_large_patch4_window12_384 - all - Fold 3 - val rmse 17.7805.pth')
pathy.append('../input/paw-models-384-large-all/swin_large_patch4_window12_384 - all - Fold 4 - val rmse 16.9883.pth')
pathy.append('../input/paw-models-384-large-all/swin_large_patch4_window12_384 - all - Fold 5 - val rmse 16.5446.pth')
pathy.append('../input/paw-models-384-large-all/swin_large_patch4_window12_384 - all - Fold 6 - val rmse 16.8143.pth')
pathy.append('../input/paw-models-384-large-all/swin_large_patch4_window12_384 - all - Fold 7 - val rmse 16.8518.pth')
pathy.append('../input/paw-models-384-large-all/swin_large_patch4_window12_384 - all - Fold 8 - val rmse 16.6582.pth')
pathy.append('../input/paw-models-384-large-all/swin_large_patch4_window12_384 - all - Fold 9 - val rmse 17.6446.pth')

In [ ]:
LOAD_SVR_FROM_PATH = '../input/swin-svm-train-384/'
from sklearn.metrics import mean_squared_error

In [ ]:
super_final_predictions = []
super_final_predictions2 = []
super_final_oof_predictions = []
super_final_oof_predictions2 = []
super_final_oof_true = []

for fold_ in range(10):
    print('#'*25)
    print('### FOLD',fold_+1)
    print('#'*25)
        
    #model.load(f"../input/paw-models/model_f{fold_}.bin", device="cuda", weights_only=True)

    #df_test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
    #df_test['path'] = [f"../input/petfinder-pawpularity-score/test/{x}.jpg" for x in df_test["Id"].values]
        
    #df_valid = df[df.kfold == fold_]
    #valid_img_paths = [f"../input/petfinder-pawpularity-score/train/{x}.jpg" for x in df_valid["Id"].values]

    dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
    ]
    
    name = f"SVR_fold_{fold_}.pkl" 
    if LOAD_SVR_FROM_PATH is None:
        ##################
        # EXTRACT TRAIN EMBEDDINGS
        
        df_train = df[df.kfold != fold_]
        #train_img_paths = [f"../input/petfinder-pawpularity-score/train/{x}.jpg" for x in df_train["Id"].values]
        
        print('Extracting train embedding...')
        embedx,_ = inference_func(df_train , pathy[fold_])
        
        ##################
        # FIT RAPIDS SVR
        print('Fitting SVR...')
        clf = SVR(C=20.0)
        clf.fit(embedx.astype('float32'), df_train.Pawpularity.values.astype('int32'))
    
        ##################
        # SAVE RAPIDS SVR 
        pickle.dump(clf, open(name, "wb"))
        
    else:
        ##################
        # LOAD RAPIDS SVR 
        print('Loading SVR...',LOAD_SVR_FROM_PATH+name)
        clf = pickle.load(open(LOAD_SVR_FROM_PATH+name, "rb"))

    ##################
    # TEST PREDICTIONS

    print('Predicting test...')
    embedt,final_test_predictions = inference_funcy(test , pathy[fold_])

    #final_test_predictions = []
   

    #final_test_predictions = [sigmoid(x) * 100 for x in final_test_predictions]
    final_test_predictions2 = clf.predict(embedt)
    super_final_predictions.append(final_test_predictions)
    super_final_predictions2.append(final_test_predictions2)
    ##################
    
    ##################

In [ ]:
best_w = 0.45
super_final_predictions = np.mean(np.column_stack(super_final_predictions), axis=1)
super_final_predictions2 = np.mean(np.column_stack(super_final_predictions2), axis=1)
test['y'] = (1-best_w)*super_final_predictions + best_w*super_final_predictions2
test.head()

# swin 224

In [ ]:
val_augz =  A.Compose(
    [ 
        A.Resize(329,320,p=1.0),
        A.CenterCrop(224,224,p=1.0),
        A.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
        ToTensorV2()
    ]
)

In [ ]:
def inference_funcz(oof , path):
    oof_dataset = Pets(oof.reset_index(drop=True),augs = val_augz)
    oof_loader = DataLoader(oof_dataset, batch_size=16, shuffle=False,  num_workers=4)
    m = Model3(False)
    m = m.to(device)
    m.load_state_dict(torch.load(path))
    m.eval()
    bar = tqdm(oof_loader)

    PREDS1 = []
    PREDS2 = []
    with torch.no_grad():
        for batch_idx, images in enumerate(bar):
            x = images.to(device)
            output1,output2 = m(x)
            output2 = output2.sigmoid()
            output2 = output2*100
            PREDS1 += [output1.detach().cpu()]
            PREDS2 += [output2.detach().cpu()]
        PREDS1 = torch.cat(PREDS1).cpu().numpy()
        PREDS2 = torch.cat(PREDS2).cpu().numpy()  
    return PREDS1,PREDS2

In [ ]:
pathz = []
pathz.append("../input/paw-models-all-5-epoch/swin_large_patch4_window7_224_in22k - all - Fold 0 - val rmse 17.5231.pth")
pathz.append('../input/paw-models-all-5-epoch/swin_large_patch4_window7_224_in22k - all - Fold 1 - val rmse 17.8166.pth')
pathz.append('../input/paw-models-all-5-epoch/swin_large_patch4_window7_224_in22k - all - Fold 2 - val rmse 17.4906.pth')
pathz.append('../input/paw-models-all-5-epoch/swin_large_patch4_window7_224_in22k - all - Fold 3 - val rmse 17.5070.pth')
pathz.append('../input/paw-models-all-5-epoch/swin_large_patch4_window7_224_in22k - all - Fold 4 - val rmse 17.4393.pth')
pathz.append('../input/paw-models-all-5-epoch/swin_large_patch4_window7_224_in22k - all - Fold 5 - val rmse 17.2695.pth')
pathz.append('../input/paw-models-all-5-epoch/swin_large_patch4_window7_224_in22k - all - Fold 6 - val rmse 17.9887.pth')
pathz.append('../input/paw-models-all-5-epoch/swin_large_patch4_window7_224_in22k - all - Fold 7 - val rmse 17.2341.pth')
pathz.append('../input/paw-models-all-5-epoch/swin_large_patch4_window7_224_in22k - all - Fold 8 - val rmse 17.0782.pth')
pathz.append('../input/paw-models-all-5-epoch/swin_large_patch4_window7_224_in22k - all - Fold 9 - val rmse 17.7036.pth')

In [ ]:
LOAD_SVR_FROM_PATH = '../input/svm-swin-train-224/'

In [ ]:
test

In [ ]:
super_final_predictions = []
super_final_predictions2 = []
super_final_oof_predictions = []
super_final_oof_predictions2 = []
super_final_oof_true = []

for fold_ in range(10):
    print('#'*25)
    print('### FOLD',fold_+1)
    print('#'*25)
        
    #model.load(f"../input/paw-models/model_f{fold_}.bin", device="cuda", weights_only=True)

    #df_test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
    #df_test['path'] = [f"../input/petfinder-pawpularity-score/test/{x}.jpg" for x in df_test["Id"].values]
        
    #df_valid = df1[df1.kfold == fold_]
    #valid_img_paths = [f"../input/petfinder-pawpularity-score/train/{x}.jpg" for x in df_valid["Id"].values]

    dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
    ]
    
    name = f"SVR_fold_{fold_}.pkl" 
    if LOAD_SVR_FROM_PATH is None:
        ##################
        # EXTRACT TRAIN EMBEDDINGS
        
        df_train = df1[df1.kfold != fold_]
        #train_img_paths = [f"../input/petfinder-pawpularity-score/train/{x}.jpg" for x in df_train["Id"].values]
        
        print('Extracting train embedding...')
        embedx,_ = inference_funcz(df_train , pathz[fold_])
        
        ##################
        # FIT RAPIDS SVR
        print('Fitting SVR...')
        clf = SVR(C=20.0)
        clf.fit(embedx.astype('float32'), df_train.Pawpularity.values.astype('int32'))
    
        ##################
        # SAVE RAPIDS SVR 
        pickle.dump(clf, open(name, "wb"))
        
    else:
        ##################
        # LOAD RAPIDS SVR 
        print('Loading SVR...',LOAD_SVR_FROM_PATH+name)
        clf = pickle.load(open(LOAD_SVR_FROM_PATH+name, "rb"))

    ##################
    # TEST PREDICTIONS

    print('Predicting test...')
    embedt,final_test_predictions = inference_funcz(test , pathz[fold_])

    #final_test_predictions = []
   

    #final_test_predictions = [sigmoid(x) * 100 for x in final_test_predictions]
    final_test_predictions2 = clf.predict(embedt)
    super_final_predictions.append(final_test_predictions)
    super_final_predictions2.append(final_test_predictions2)
    ##################
    
    ##################

In [ ]:
best_w = 0.55
super_final_predictions = np.mean(np.column_stack(super_final_predictions), axis=1)
super_final_predictions2 = np.mean(np.column_stack(super_final_predictions2), axis=1)
test['z'] = (1-best_w)*super_final_predictions + best_w*super_final_predictions2
test.head()

In [ ]:
test['xx'] = (test.x+test.z+test.y)/3.

In [ ]:
test["Pawpularity"] = -1.
test["Pawpularity"] = test["Pawpularity"].astype(np.float)
test.head()

In [ ]:
if(len(cats) > 0):
    for index, row in cats.iterrows():
        test.loc[test['Id'] == row['Id'], 'Pawpularity'] = row['Pawpularity']


In [ ]:
if(len(dogs_unk) > 0):
    for index, row in dogs_unk.iterrows():
        test.loc[test['Id'] == row['Id'], 'Pawpularity'] = row['Pawpularity']

# Weighted Averaging different models

In [ ]:
test["Pawpularity"] =  0.45*test["Pawpularity"] + 0.55*test['xx']

In [ ]:
sub = test[["Id" , "Pawpularity"]]

In [ ]:
sub["Pawpularity"] = 0.5*sub["Pawpularity"] + 0.5*df_test["Pawpularity"]

In [ ]:
sub.to_csv("submission.csv", index = False)

In [ ]:
sub